<a href="https://colab.research.google.com/github/bahti21/tech_specifications_for_work/blob/main/%D0%A5%D0%B0%D0%BC%D0%B8%D1%82%D0%BE%D0%B2_%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_BWG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Отчет по данным.

Ошибки в файле, котрые нужно и исправить:  
1) Общий формат в колонках с E (показы) по I (заявка), нужно перевести в числовой формат, кроме столбца "Расходы" (тут остается денежный).  
2) заполнить пустые ячейки нулями.  
3) в середине таблицы (с 1003 строчки) есть момент когда столбец  равен 0, но у кликов есть значения, что по сути не должно быть, так как без показа нельзя кликнуть на рекламу (я так понял). Тут я бы уточнил, как именно собирались данные, и наверное можно пометить эти строки, либо убрать их, так как при сборке метрики у коллег будет статистическая ошибка, при рассчетах CPA и CTR.

Сводная таблица (без учета ошибок):

In [86]:
dannye_df = pd.read_excel('/content/Новое тестовое  задание.xlsx', sheet_name='Данные') #в ковычки добавляем путь к файлу, который загружен в колаб.
dannye_df.head()
pivot_table = dannye_df.pivot_table(
    index=['Месяц', 'Рекламный канал'],
    values=['Показы', 'Клики', 'Расход с НДС и АК', 'Регистрация', 'Заявка'],
    aggfunc='sum',
    fill_value=0
).reset_index()

pivot_table['CTR'] = (pivot_table['Клики'] / pivot_table['Показы']) * 100
pivot_table['CPA Регистрация'] = pivot_table['Расход с НДС и АК'] / pivot_table['Регистрация']
pivot_table['CPA Заявка'] = pivot_table['Расход с НДС и АК'] / pivot_table['Заявка']
pivot_table.replace([float('inf'), -float('inf')], 0, inplace=True)
pivot_table.fillna(0, inplace=True)
pivot_table


,Месяц,Рекламный канал,Заявка,Клики,Показы,Расход с НДС и АК,Регистрация,CTR,CPA Регистрация,CPA Заявка
0,Декабрь,Google Ads,82,17070,1277136,184128.022424,343,1.336584,536.816392,2245.463688
1,Декабрь,MyTarget,3,2036,461882,50000.004000,27,0.440805,1851.852000,16666.668000
2,Декабрь,TG Ads,1,15069,2753685,621000.000000,459,0.547230,1352.941176,621000.000000
3,Декабрь,ВКонтакте,9,32580,5558336,424816.270000,323,0.586147,1315.220650,47201.807778
4,Декабрь,Яндекс.Директ,63,33699,11597467,428396.722440,509,0.290572,841.643855,6799.947975
5,Ноябрь,Google Ads,81,19943,1508564,164960.004049,519,1.321986,317.842012,2036.543260
6,Ноябрь,MyTarget,0,3193,780116,66666.996000,17,0.409298,3921.588000,0.000000
7,Ноябрь,TG Ads,2,15980,2298097,530935.189600,420,0.695358,1264.131404,265467.594800
8,Ноябрь,ВКонтакте,10,33586,7441839,442160.360000,333,0.451313,1327.808889,44216.036000
9,Ноябрь,Яндекс.Директ,112,30915,7308801,461498.990160,615,0.422983,750.404862,4120.526698


1. Этот код для заполнения пустых ячеек нулями

In [68]:
import pandas as pd
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name, sheet_name='Sheet1')
df.fillna(0, inplace=True)

output_file_name = 'Новое тестовое  задание1.xlsx'
df.to_excel(output_file_name, index=False)
#скачиваем новый файл
files.download(output_file_name)

Saving Новое тестовое  задание1.xlsx to Новое тестовое  задание1 (2).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2. Данный код нужен для исправления ячеек из "общего" формата в формат "числовой".

In [69]:
import pandas as pd
from google.colab import files
#скачиваем новый файл, полученный выше "Новое тестовое  задание1.xlsx"
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name, sheet_name = 'Sheet1')
numeric_columns = ['Показы', 'Клики', 'Регистрация', 'Заявка']
#числовой формат (по сути должно работать, у меня почему-то также остается общий формат ячеек, возможно из-за пиратского экселя)
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.to_excel(file_name, index=False)
output_file_name = 'Новое тестовое  задание2.xlsx'
df.to_excel(output_file_name, index=False)
files.download(output_file_name)

Saving Новое тестовое  задание1.xlsx to Новое тестовое  задание1 (3).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ну и последний код, котороый убирает строки если колчисетво показов = 0, а количество кликов не равно 0.

In [79]:
import pandas as pd
from google.colab import files
from openpyxl import load_workbook
#снова скачиваем файл который получили выше, который уже под номером 2
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
xls = pd.ExcelFile(file_name)
sheet_name = 'Sheet1'
df = pd.read_excel(file_name, sheet_name=sheet_name)
df = df[~((df['Показы'] == 0) & (df['Клики'] != 0))] #можно добавить условие, чтобы количество кликов было меньше или равно количеству показов, но просмотрев файл такого не обнаружил поэтому не стал добавлять
df.to_excel(file_name, index=False)
output_file_name = 'Новое тестовое  задание3.xlsx'
df.to_excel(output_file_name, index=False)
files.download(output_file_name)


Saving Новое тестовое  задание2.xlsx to Новое тестовое  задание2 (3).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Готовая сводная таблица с учетом ошибок.

In [87]:
dannye_df = pd.read_excel('/content/Новое тестовое  задание3.xlsx', sheet_name='Sheet1') #в ковычки добавляем путь к файлу который получили последним, но перед этим добавим его в колаб.
dannye_df.head()
pivot_table = dannye_df.pivot_table(
    index=['Месяц', 'Рекламный канал'],
    values=['Показы', 'Клики', 'Расход с НДС и АК', 'Регистрация', 'Заявка'],
    aggfunc='sum',
    fill_value=0
).reset_index()

pivot_table['CTR'] = (pivot_table['Клики'] / pivot_table['Показы']) * 100
pivot_table['CPA Регистрация'] = pivot_table['Расход с НДС и АК'] / pivot_table['Регистрация']
pivot_table['CPA Заявка'] = pivot_table['Расход с НДС и АК'] / pivot_table['Заявка']
pivot_table.replace([float('inf'), -float('inf')], 0, inplace=True)
pivot_table.fillna(0, inplace=True)
pivot_table

,Месяц,Рекламный канал,Заявка,Клики,Показы,Расход с НДС и АК,Регистрация,CTR,CPA Регистрация,CPA Заявка
0,Декабрь,Google Ads,82,17070,1277136,184128.022424,343,1.336584,536.816392,2245.463688
1,Декабрь,MyTarget,3,2036,461882,50000.004000,27,0.440805,1851.852000,16666.668000
2,Декабрь,TG Ads,1,15069,2753685,621000.000000,459,0.547230,1352.941176,621000.000000
3,Декабрь,ВКонтакте,9,32580,5558336,424816.270000,323,0.586147,1315.220650,47201.807778
4,Декабрь,Яндекс.Директ,63,33699,11597467,428396.722440,509,0.290572,841.643855,6799.947975
5,Ноябрь,Google Ads,81,19943,1508564,164960.004049,519,1.321986,317.842012,2036.543260
6,Ноябрь,MyTarget,0,3193,780116,66666.996000,17,0.409298,3921.588000,0.000000
7,Ноябрь,TG Ads,2,15980,2298097,530935.189600,420,0.695358,1264.131404,265467.594800
8,Ноябрь,ВКонтакте,10,33586,7441839,442160.360000,333,0.451313,1327.808889,44216.036000
9,Ноябрь,Яндекс.Директ,112,30915,7308801,461498.990160,615,0.422983,750.404862,4120.526698


Сложности по заданию:   
1) Достаточно много времени ушло на разбор синтаксиса в библиотеке openpyxl для работы с Excel.  
2) Код можно оптимизировать и сделать все действия  в одном и скачать в конце 1 файл.   
Вообще делал так, на случай если будет ошибка, чтоб понимать на каком из этапов всё сломалось.   
3) Итогововая сводная таблица с учетом ошибок выводит те же числа, что и таблица без учета ошибок. Не понял в чем проблема.